# Autograd: automatic differentiation

> Author : Badr TAJINI - Machine Learning 2 & Deep learning - ECE 2025-2026

---

When executing tensor operations, PyTorch can automatically construct on-the-fly the graph of operations to compute the gradient of any quantity with respect to any tensor involved.

More information on autotomatic differentiation in Pytorch [here](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py)

## Visualizing the computational graph

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch

PyTorchViz is a small package to create visualizations of PyTorch execution graphs and traces.

In [ ]:
!pip install torchviz


In [ ]:
import torchviz

Inn order to enable automatic differentiation, PyTorch keeps track of all operations involving tensors for which the gradient may need to be computed (i.e., require_grad is True). The operations are recorded as a directed graph. 

Torch tensors are recalled [here](https://pytorch.org/docs/stable/tensors.html) 



In [ ]:
x = torch.ones(2, 2)
print(x)

A Tensor has a Boolean field *requires_grad*, set to False by default, which states if PyTorch should build the graph of operations so that gradients wrt to it can be computed.

In [ ]:
x.requires_grad

In [ ]:
x.numpy()

In [ ]:
x.requires_grad_(True)
x.requires_grad

### Question: The following cell gives an error. Why?

In [ ]:
x.numpy()

The detach() method constructs a new view on a tensor which is declared not to need gradients, i.e., it is to be excluded from further tracking of operations, and therefore the subgraph involving this view is not recorded.

In [ ]:
x.detach().numpy()

In [ ]:
x.data.numpy() # Torch.data returns the "data" (array_like) stored in the tensor

In [ ]:
x.requires_grad

In [ ]:
y = x + 2
print(y)
torchviz.make_dot(y)

**Broadcasting**

Recall: Broadcasting **automatically** expands dimensions by replicating coefficients, when it is necessary to perform operations.

1. If one of the tensors has fewer dimensions than the other, it is reshaped by adding as many dimensions of size 1 as necessary in the front; then
2. for every mismatch, if one of the two tensor is of size one, it is expanded along this axis by replicating  coefficients.

If there is a tensor size mismatch for one of the dimension and neither of them is one, the operation fails.

In [ ]:
A = torch.tensor([[1.], [2.], [3.], [4.]])
print(A.size())
print(A)
B = torch.tensor([[5., -5., 5., -5., 5.]])
print(B.size())
print(B)
C = A + B
print(C)

Back to Autograd!

In [ ]:
x = torch.ones(2, 2)
x.requires_grad_(True)
y = x+2
print(y)

In [ ]:
y.requires_grad

Tensor and Function are interconnected and build up an acyclic graph, that encodes a complete history of computation. 

Each tensor has a .grad_fn attribute that references a Function that has created the Tensor (except for Tensors created by the user - their grad_fn is None).

In [ ]:
print(x)
print(x.grad_fn)
print(y)
print(y.grad_fn)
torchviz.make_dot(y) 

In [ ]:
z = y * y * 3
print(z)
torchviz.make_dot(z) 

In [ ]:
out = z.mean()
print(out)
torchviz.make_dot(out) 

After the computation is finished, i.e. _forward pass_, you can call ```.backward()``` and have all the gradients computed automatically.

In [ ]:
out.backward()

The gradients w.r.t. this variable is accumulated into ```.grad```.

In [ ]:
print(x.grad)

### Question: Explain mathematically the value of "x.grad"


[Why can I not see .grad of an intermediate variable?](https://discuss.pytorch.org/t/why-cant-i-see-grad-of-an-intermediate-variable/94)

Gradients are only retained for leaf variables. 

Non-leaf variables’ gradients are not retained to be inspected later. This was done by design, to save memory.

In [ ]:
print(y.grad)
print(out.grad)

In [ ]:
x = 2 * torch.ones(2, 2)
print(x)
x.requires_grad_(True)
out = (x * x * x + 2 ).mean()
print(out)
torchviz.make_dot(out)

In [ ]:
out.backward(torch.tensor(2.0))

### Question: Explain mathematically the value of "x.grad" in the following cell

In [ ]:
print(x.grad)

## How to compute the gradient when I want?



In [ ]:
x = torch.ones(2, 2)
x.requires_grad_(True)
y = x+2
z = 3 * y ** 2 
out = z.mean()

out.backward(retain_graph=True)

print(x.grad)

Calling again the "backward" process does not launch an error

In [ ]:
out.backward(retain_graph=True)

In [ ]:
print(x.grad)

In [ ]:
torch.autograd.grad(out, out, retain_graph=True)

In [ ]:
torch.autograd.grad(out, z, retain_graph=True)

In [ ]:
torch.autograd.grad(out, y, retain_graph=True)

In [ ]:
out.backward(torch.tensor(2.0), retain_graph=True)

In [ ]:
print(x.grad)

## A last example

In [ ]:
x=torch.ones(10, requires_grad=True)

y=x**2
z=x**3
r=(y+z).sum()

r.backward()
print(x.grad)
torchviz.make_dot(r) 

## Gradient accumulation


The gradients must be set to zero manually. Otherwise they will cumulate across several _.backward()_ calls. 
This accumulating behavior is desirable in particular to compute the gradient of a loss summed over several “mini-batches,” or the gradient of a sum of losses.


In [ ]:
x = torch.ones(1, 1,requires_grad = True)
out = (x+1)**2
print(out)

In [ ]:
out.backward(torch.ones(1,1),retain_graph=True)
print(x.grad)
out.backward(2 * torch.ones(1,1),retain_graph=True)
print(x.grad)
out.backward(retain_graph=True)
print(x.grad)

In [ ]:
# Manually zero the gradients after updating weights
x.grad.data.zero_()

In [ ]:
out.backward(torch.ones(1,1),retain_graph=True)
print(x.grad)
x.grad.data.zero_()
out.backward(2 * torch.ones(1,1),retain_graph=True)
print(x.grad)
x.grad.data.zero_()
out.backward(retain_graph=True)
print(x.grad)


If you want to come back to the difference between detach and data see [Differences between .data and .detach](https://github.com/pytorch/pytorch/issues/6990)

# Playing with a neural network in pytorch: non-linear regression

## Non-linear regression with a neural network

An implementation of **(Batch) Gradient Descent** using the nn package. Here we have a super simple model with only one layer and no activation function!

Generate the dataset and visualize it.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np


# Make data
num_samples = 10
X1 = np.linspace(-5, 5, num_samples) 
X2 = np.linspace(-5, 5, num_samples) 
X1grid, X2grid = np.meshgrid(X1, X2)
Y = -(0.1*X1grid**2 + 3*np.sin(0.1*X2grid) + 0.1) 

#print(X1grid)
#print(X2grid)
#print(Y)

# Plot the surface.
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X1grid, X2grid, Y, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

In [ ]:
x1n = X1grid.ravel().reshape(-1, 1)
x2n = X2grid.ravel().reshape(-1, 1)
xn = np.concatenate((x1n,x2n),axis=1)
yn = Y.ravel().reshape(-1, 1)
#print(x1n.shape)
#print(x2n.shape)
print(xn.shape)
print(yn.shape)

# Convert in pytorch tensors
x = torch.from_numpy(xn).float()
y = torch.from_numpy(yn).float()
#print(x)
#print(y)
print(x.shape)

## Neural network with full implementation (gradient computation and estimate update)

In [ ]:
import torch

dtype = torch.float


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = num_samples**2, 2, 100, 1

# Randomly initialize weights
w1 = torch.randn(D_in, H, dtype=dtype)
w2 = torch.randn(H, D_out,  dtype=dtype)

learning_rate = 1e-6 
for t in range(10001):
    # Forward pass: compute predicted y
    h = x.mm(w1)
    
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    #print(y_pred)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 1000 == 0:
        print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

Compare the first values

In [ ]:
print(y[0:5])
print(y_pred[0:5])

Plot the approximation in 1D

In [ ]:
def plotApproximation(y,y_pred):
  plt.figure(figsize=(6, 4))
  plt.plot(y.detach().numpy(),'b-')
  plt.plot(y_pred.detach().numpy(),'r-*')
  plt.xlabel('$x$')
  plt.ylabel('$y$')
  plt.title('Approximation with the neural network')
  plt.show()

In [ ]:
plotApproximation(y,y_pred)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(y.numpy()-y_pred.numpy())
plt.title('Residuals')
plt.xlabel('$x$')
plt.ylabel('$y-\hat{y}$')
plt.show()

Last step, we use directly the optim package to update the weights and bias.

## Neural network with automatic gradient computation (but still the explicit estimate update)


A fully-connected ReLU network with one hidden layer, trained to predict y from x
by minimizing squared Euclidean distance.

This implementation uses the nn package from PyTorch to build the network.
PyTorch autograd makes it easy to define computational graphs and take gradients,
but raw autograd can be a bit too low-level for defining complex neural networks;
this is where the nn package can help. The nn package defines a set of Modules,
which you can think of as a neural network layer that has produces output from
input and may have some trainable weights.

### Question: use autograd to compute the gradient. You can start from the full implementation of the neural network.

In [ ]:
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = num_samples**2, 2, 100, 1

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(10001):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 1000 == 0:
        print(t, loss.item())

    
    # COMPLETE THE CODE FROM HERE
    # ...

In [ ]:
plotApproximation(y,y_pred)

## Neural network with automatic gradient computation and automatic estimate update

A fully-connected ReLU network with one hidden layer, trained to predict y from x
by minimizing squared Euclidean distance.

This implementation uses the nn package from PyTorch to build the network.

Rather than manually updating the weights of the model as we have been doing,
we use the optim package to define an Optimizer that will update the weights
for us. The optim package defines many optimization algorithms that are commonly
used for deep learning, including SGD+momentum, RMSProp, Adam, etc.

### Question: use the Torch optimizer to do the gradient descent. You can start from the full implementation of the neural network with automatic differentiation.

Details on Optimizer are [here](https://pytorch.org/docs/stable/optim.html)

In [ ]:
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = num_samples**2, 2, 100, 1

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-6
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
for t in range(10001):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 1000 == 0:
        print(t, loss.item())

    # COMPLETE THE CODE FROM HERE
    # ...

In [ ]:
plotApproximation(y,y_pred)